![April 31st is a perfectly fine date in Grumpy Cat's data entry system](https://github.com/snorehorse/sas-demos/raw/master/program-flow-and-error-checking/images/april_31st.jpg)

# Simplified data quality checks and program flow in SAS programs
## Part 1 (The first steps)

*A walk-through of ETL design patterns, naming conventions and simple macros. *

Based on the Spring 2017 GVSUG presentation by [Louis Herczeg](https://www.linkedin.com/in/louisherczeg/ "LinkedIn") & [Matthew Giglia](https://www.linkedin.com/in/matthewgiglia/ "LinkedIn"). 

### Real world data
Common problems include:
* typos
* duplicate records
* missing values
* business logic violations

### This demo

1. Create a combination of "good" and "bad" data. 
2. Create quality checks.
3. Automatically summarize the results.
4. Route program flow based on the summary:
    - Bad data encountered: Notify stakeholders. Cease processing. Correct the problems in the source system. Run the program again.   
    - Everything's good: Do scary things. Load transformed data into a new system. Email bigwigs.

## Step 01 - Demo data

Data sets are prefixed in the project so that they'll sort in order of creation when viewed in the library or with `proc contents`. "s01" is step 01, and will appear as the top entry.

In [1]:
options nosource nonotes;

/* Get some good data */
data s01_test_data;
    set sashelp.cars(keep=make model Type drivetrain Origin MSRP Cylinders);
        model = left(model);
run;

/* Insert bad data */
proc sql;
    insert into s01_test_data

    /* Bad cylinder counts */
    values ("Foo", "RAV5", "SUV", "Asia", "Front", 25000, -4) 
    values ("Foo", "RAV8a", "SUV", "Asia", "Front", 31000, 80)
    values ("Foo", "RAV8b", "SUV", "Asia", "Front", 31000, 3.5)

    /* Bad MSRP */
    values ("Foo", "Bar", "SUV", "Asia", "Front", 58, 6)
    values ("Fizz", "Buzz", "SUV", "Asia", "Front", -25000, 6)
    values ("Fizz", "Snorehorse", "SUV", "Asia", "Front", ., 6)

    /* Duplicate record based on make, model, drivetrain */
    values ("Saab", "9-5 Aero", "Wagon", "Europe", "Front", 50000, 4) 

    /* Bad Type */
    values ("Snorehorse", "Townlander", "BadValue", "USA", "Front", 40000, 4) 

    /* Bad Drive Train */
    values ("Snowplow", "Highlander", "Wagon", "USA", "Apple", 45000, 4) 

    /* MSRP outlier */
    values ("Llama", "Loaf", "Wagon", "USA", "Front",  1000000, 4) 
    ;
quit;

title "Data Extract"; footnote 'Simulated bad data starts on row 429.';
proc print data=s01_test_data(firstobs=426); run; title; footnote;

Obs,Make,Model,Type,Origin,DriveTrain,MSRP,Cylinders
426,Volvo,S80 T6 4dr,Sedan,Europe,Front,"$45,210",6.0
427,Volvo,V40,Wagon,Europe,Front,"$26,135",4.0
428,Volvo,XC70,Wagon,Europe,All,"$35,145",5.0
429,Foo,RAV5,SUV,Asia,Front,"$25,000",-4.0
430,Foo,RAV8a,SUV,Asia,Front,"$31,000",80.0
431,Foo,RAV8b,SUV,Asia,Front,"$31,000",3.5
432,Foo,Bar,SUV,Asia,Front,$58,6.0
433,Fizz,Buzz,SUV,Asia,Front,"$-25,000",6.0
434,Fizz,Snorehorse,SUV,Asia,Front,.,6.0
435,Saab,9-5 Aero,Wagon,Europe,Front,"$50,000",4.0


## Step 02 - Write an arbitrary number of quality checks

*Note: In Part 2, these will be wrapped in macros, for reuse throughout the program. *

- Each check gets its own data set

- Notice the naming convention, "\_CHK\_"! This allows the checks to be automatically summarized.

- The data set labels will provide the description of the check, on the summary table.

#### Check cylinder count

In [2]:
%let lbl = '01-Cylinders must be between 3 and 12, and whole numbers';

data s02_chk_01_BAD_CYLINDERS(Label=&lbl);
    length check_description $100;
    SET s01_test_data;
    check_description=&lbl.;
    WHERE not((3 <=cylinders <=12) and (floor(cylinders)=cylinders));
RUN;

proc print; run;

Obs,check_description,Make,Model,Type,Origin,DriveTrain,MSRP,Cylinders
1,"01-Cylinders must be between 3 and 12, and whole numbers",Mazda,RX-8 4dr automatic,Sports,Asia,Rear,"$25,700",.
2,"01-Cylinders must be between 3 and 12, and whole numbers",Mazda,RX-8 4dr manual,Sports,Asia,Rear,"$27,200",.
3,"01-Cylinders must be between 3 and 12, and whole numbers",Foo,RAV5,SUV,Asia,Front,"$25,000",-4.0
4,"01-Cylinders must be between 3 and 12, and whole numbers",Foo,RAV8a,SUV,Asia,Front,"$31,000",80.0
5,"01-Cylinders must be between 3 and 12, and whole numbers",Foo,RAV8b,SUV,Asia,Front,"$31,000",3.5


Looks like we detected some bad data in the original `SASHELP.CARS`!

#### Check MSRP bad values

In [3]:
%let lbl = '02-MSRP must be greater than $5000';

data s02_chk_02_BAD_MSRP(Label=&lbl);
    length check_description $100;
    SET s01_test_data;
    check_description=&lbl.;
    WHERE not(MSRP >=5000);
RUN;

proc print; run;

Obs,check_description,Make,Model,Type,Origin,DriveTrain,MSRP,Cylinders
1,02-MSRP must be greater than $5000,Foo,Bar,SUV,Asia,Front,$58,6
2,02-MSRP must be greater than $5000,Fizz,Buzz,SUV,Asia,Front,"$-25,000",6
3,02-MSRP must be greater than $5000,Fizz,Snorehorse,SUV,Asia,Front,.,6


#### Check for dupes

In [4]:
%let lbl = '03-Must be unique on (make model drivetrain)';

proc sort nouniquekey data=s01_test_data uniqueout=_null_ 
        out=s02_chk_03_NOT_UNIQUE(Label=&lbl);
    by make model drivetrain;
run;

/* Add the description */
data s02_chk_03_NOT_UNIQUE(Label=&lbl);
    length check_description $100;
    SET s02_chk_03_NOT_UNIQUE;
    check_description=&lbl.;
RUN;

proc print; run;

Obs,check_description,Make,Model,Type,Origin,DriveTrain,MSRP,Cylinders
1,03-Must be unique on (make model drivetrain),Saab,9-5 Aero,Wagon,Europe,Front,"$40,845",4
2,03-Must be unique on (make model drivetrain),Saab,9-5 Aero,Wagon,Europe,Front,"$50,000",4


#### Check for MSRP outliers

In [5]:
%let lbl = '04- Extreme MSRP outliers. The MSRP output is standard deviations from the mean MSRP.';

PROC STANDARD DATA=s01_test_data MEAN=0 STD=1 
        OUT=s02_chk_04_msrp_outlier(label=&lbl. where=(msrp ge 5)); *Detect anything 5 standard deviations from normal;
    format msrp 6.3;
    VAR msrp;
RUN;

/* Add the desciption*/
data s02_chk_04_msrp_outlier(Label=&lbl);
    length check_description $100;
    SET s02_chk_04_msrp_outlier;
    check_description=&lbl.;
RUN;

proc print; run;

Obs,check_description,Make,Model,Type,Origin,DriveTrain,MSRP,Cylinders
1,04- Extreme MSRP outliers. The MSRP output is standard deviations from the mean MSRP.,Llama,Loaf,Wagon,USA,Front,19.216,4


## Step 03 - Automatically gather and summarize the bad rows

See that by keeping to a naming convention above, any arbitrary number of checks can be added with the same summary effort. No inspecting individual data sets by hand!

### Get the row count for each check

In [6]:
/* Create a table to store the summarized results */
proc sql ;
    create table s03_summarize_checks
            (description char(256), tbl char(32), count num);
quit;

/*  Do a fuzzy match for the data sets matching our naming convention. 
    Here's where the data set label is picked up and becomes the description.
*/
proc sql ;
    insert into s03_summarize_checks select memlabel, memname, nobs from 
        dictionary.tables where libname eq "WORK" and memname like "%^_CHK^_%" escape '^';
quit;

title 'Summary of the checks';
proc print data=s03_summarize_checks; run; title;

Obs,description,tbl,count
1,"01-Cylinders must be between 3 and 12, and whole numbers",S02_CHK_01_BAD_CYLINDERS,5
2,02-MSRP must be greater than $5000,S02_CHK_02_BAD_MSRP,3
3,03-Must be unique on (make model drivetrain),S02_CHK_03_NOT_UNIQUE,2
4,04- Extreme MSRP outliers. The MSRP output is standard deviations from the mean MSRP.,S02_CHK_04_MSRP_OUTLIER,1


#### About the DICTIONARY.TABLES above:

This view contains information about all the data sets your session knows about. Watch out! In big corporate environments, that's a lot of data sets! ** Avoid doing a `select * from DICTIONARY.TABLES`... it might take forever to run. **
See [this pdf](http://www2.sas.com/proceedings/sugi30/070-30.pdf) for additional details and examples.

#### Note on the SQL fuzzy matching above:

- In ANSI and `PROC SQL`, `LIKE` is used for wild-card searching.

    - `%` matches any zero or more characters

    - `_` matches any single character

- In our naming convention, the underscores in `_CHK_` need to be escaped in order to take them literally, because underscores are part of the `LIKE` logic, so it becomes `%^_CHK^_%`.

### Get the total number of bad rows

In [7]:
proc sql noprint;
    select sum(count) into :sum_bad_rows from s03_summarize_checks;
quit;
%put The total number of bad rows is: &sum_bad_rows.;

The above code gets the sum of the summary table's count column, and stores the result in a macro variable we're calling `sum_bad_rows`. The variable is then available for use throughout the program. 

### Route program flow

You could use it as the determiner for having your program do one set of things if there are no problems, and another set of things otherwise. Below is a basic example of this:

In [8]:
%macro decide_what_to_do();
    %if &sum_bad_rows. eq 0 %then 
        %put Doing scary things like email stakeholders, commit to production, sftp to third parties, etc;
    %else
        %put ERROR: Bad rows detected. Fix data in source system and try again. Keep your job another day.;
%mend decide_what_to_do;

%decide_what_to_do();

### Step 04 - Get a detailed summary of bad rows

Maybe one of the things to do when bad data is detected is to do a detailed summary. Let's grab the checks' data set names from the summary table:

In [9]:
proc sql noprint;
    select tbl into :check_tbls separated by ' ' from s03_summarize_checks;
quit;
%put Data quality check tables are: &check_tbls.;

In [10]:
/*  Append all the check data sets together.

    For those unfamiliar with the macro language, imagine replacing &check_tbls. below with the contents of the macro 
    variable (as seen above)... and then executing the data step with the newly overwritten text. So the data step is executed
    with this middle line:
    
        set S02_CHK_01_BAD_CYLINDERS S02_CHK_02_BAD_MSRP S02_CHK_03_NOT_UNIQUE S02_CHK_04_MSRP_OUTLIER;
        
*/
data s04_gather_bad_rows;
    set &check_tbls.;
run;

proc print; run;

Obs,check_description,Make,Model,Type,Origin,DriveTrain,MSRP,Cylinders
1,"01-Cylinders must be between 3 and 12, and whole numbers",Mazda,RX-8 4dr automatic,Sports,Asia,Rear,"$25,700",.
2,"01-Cylinders must be between 3 and 12, and whole numbers",Mazda,RX-8 4dr manual,Sports,Asia,Rear,"$27,200",.
3,"01-Cylinders must be between 3 and 12, and whole numbers",Foo,RAV5,SUV,Asia,Front,"$25,000",-4.0
4,"01-Cylinders must be between 3 and 12, and whole numbers",Foo,RAV8a,SUV,Asia,Front,"$31,000",80.0
5,"01-Cylinders must be between 3 and 12, and whole numbers",Foo,RAV8b,SUV,Asia,Front,"$31,000",3.5
6,02-MSRP must be greater than $5000,Foo,Bar,SUV,Asia,Front,$58,6.0
7,02-MSRP must be greater than $5000,Fizz,Buzz,SUV,Asia,Front,"$-25,000",6.0
8,02-MSRP must be greater than $5000,Fizz,Snorehorse,SUV,Asia,Front,.,6.0
9,03-Must be unique on (make model drivetrain),Saab,9-5 Aero,Wagon,Europe,Front,"$40,845",4.0
10,03-Must be unique on (make model drivetrain),Saab,9-5 Aero,Wagon,Europe,Front,"$50,000",4.0


## Next steps

Imagine multiple groups of checks, based on the stage you're at in processing the data. Maybe you want some checks to print `ERROR` to the log, while others are just `WARNING`. If a different naming convention was used for each, separate summaries could be generated. 

Macros could be used to make the fuzzy search and summary gathering reusuable. Just give it your naming convention and it will automatically process. 

Again macros could be used to create a reusable series of checks. Maybe the first pass would be to detect the bad data, while the 2nd pass is after a manual corrections file has been loaded.

More demos to come, expanding on these ideas!